## check or install cuda version, e.g.,conda install cudatoolkit=10.1
```
##pip install cellpose
pip uninstall mxnet-mkl
pip uninstall mxnet
pip install mxnet-cu101 
conda install -c anaconda ipython
```
### imjoy
```
pip install imjoy
pip install -U imjoy-jupyter-extension

```

#### This is a magic command to run using imjoy api without export ImjoyPlugin, after first run
```
await asyncio.ensure_future(run())
```

In [ ]:
from imjoy import api
import numpy as np
import asyncio
# import imageio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src="https://kaibu.org/#/app")

        image = np.random.randint(0, 255, [500, 500], dtype='uint8')

        # view image
        await viewer.view_image(image, type="itk-vtk", name="random pixels")

        points = np.random.randint(0, 500, [100, 2], dtype='uint16')
        layer = await viewer.add_points(points, face_color="red")

        async def say_hello():
            await api.alert('Hello!')

        async def get_geojson_features():
            # get the annotation in geojson format
            features = await layer.get_features()
            await api.alert(str(features))

        await viewer.set_ui({"title": "Utilities",
                             "elements": [
                                 {"_rintf": True,
                                  "type": "button",
                                  "label": "Say Hello",
                                  "callback": say_hello
                                 },
                                 {"_rintf": True,
                                  "type": "button",
                                  "label": "Show GeoJSON",
                                  "callback": get_geojson_features
                                 },

                             ]
                            })
api.export(ImJoyPlugin())